In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


In [2]:
trials=100_00_00       #total simulated events
low=-1;high=1;interval=high-low
test=interval*np.random.rand(trials,3)+low
#test
x=test[:,0]
y=test[:,1]
z=test[:,2]

In [3]:
x

array([-0.79904787,  0.08967453, -0.83482218, ..., -0.45231481,
        0.5463    , -0.8096832 ])

In [4]:
y

array([ 0.91294816, -0.98654293,  0.0244526 , ...,  0.61600121,
       -0.34169503,  0.04360177])

In [5]:
z

array([-0.64016486, -0.1001478 , -0.67056954, ..., -0.26322895,
        0.53330233, -0.23771131])

In [6]:

r=np.sqrt(x**2+y**2+z**2)
r

array([1.37177363, 0.99565961, 1.07106933, ..., 0.80829182, 0.83642726,
       0.84498205])

In [7]:
test=np.column_stack((test,r))

In [8]:
df=pd.DataFrame(test,columns=['x','y','z','r']) #data frame with x,y,z and r
df.head()

,x,y,z,r
0,-0.799048,0.912948,-0.640165,1.371774
1,0.089675,-0.986543,-0.100148,0.995660
2,-0.834822,0.024453,-0.670570,1.071069
3,-0.332600,-0.663478,-0.689743,1.013199
4,0.562297,-0.003648,0.957103,1.110062


In [9]:
df_less=df[df.r<0.5]          #data frame with r<0.5 (radius of the bead)
df_less.head()
total_counts=df_less.shape[0]
total_counts

65218

In [10]:
# %matplotlib inline
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.scatter(df_less.x,df_less.y,df_less.z)

In [11]:
#this function give the normal distance from a point to the surface of the sphere

def get_upward_distance(x,y,z,dz=0.001,r=0.5):
    '''
    returns normal distance from a point in the sphere to the surface as well as the z co-ordinate in the sphere
    '''
    #z=np.abs(z)
    d=np.sqrt(x**2+y**2+z**2)
    if z==-0.5: return 1.0,0.5
    z_up=0
    #print(f"before while z_up: {z_up}")
    while(d<r):
        z+=dz
        #print(f"Inside while z: {z}")
        z_up+=dz
        d=np.sqrt(x**2+y**2+z**2)
    #print(f"after while z_up: {z_up}")
    return z_up,z
        
    

In [12]:
#distance travelled in the bead

df_less['z_new']=df_less.apply(lambda x:get_upward_distance(x['x'],x['y'],x['z'])[1],axis=1)
df_less['d_bead']=df_less.apply(lambda x:get_upward_distance(x['x'],x['y'],x['z'])[0],axis=1)
df_less.head()

/tmp/ipykernel_107330/2709360843.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_less['z_new']=df_less.apply(lambda x:get_upward_distance(x['x'],x['y'],x['z'])[1],axis=1)
/tmp/ipykernel_107330/2709360843.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_less['d_bead']=df_less.apply(lambda x:get_upward_distance(x['x'],x['y'],x['z'])[0],axis=1)


,x,y,z,r,z_new,d_bead
11,0.307446,0.109184,-0.214118,0.390245,0.379882,0.594
24,0.056229,0.177338,0.117494,0.220035,0.464494,0.347
39,-0.208893,0.292758,-0.097183,0.372543,0.347817,0.445
46,-0.359449,-0.203552,0.130965,0.433346,0.281965,0.151
68,0.176059,-0.111991,0.265345,0.337560,0.455345,0.190


In [13]:
#distance travelled in the button
button_thickness=3.0
df_less['d_button']=button_thickness/2.-df_less['z_new'] #Thickness of button is 3 mm and thickness above 0 is 1.5 mm
df_less.head()

/tmp/ipykernel_107330/1364162647.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_less['d_button']=button_thickness/2.-df_less['z_new'] #Thickness of button is 3 mm and thickness above 0 is 1.5 mm


,x,y,z,r,z_new,d_bead,d_button
11,0.307446,0.109184,-0.214118,0.390245,0.379882,0.594,1.120118
24,0.056229,0.177338,0.117494,0.220035,0.464494,0.347,1.035506
39,-0.208893,0.292758,-0.097183,0.372543,0.347817,0.445,1.152183
46,-0.359449,-0.203552,0.130965,0.433346,0.281965,0.151,1.218035
68,0.176059,-0.111991,0.265345,0.337560,0.455345,0.190,1.044655


In [14]:
#changing mm to cm
mm_to_cm=0.1
df_less=df_less*mm_to_cm
df_less.head()

,x,y,z,r,z_new,d_bead,d_button
11,0.030745,0.010918,-0.021412,0.039024,0.037988,0.0594,0.112012
24,0.005623,0.017734,0.011749,0.022004,0.046449,0.0347,0.103551
39,-0.020889,0.029276,-0.009718,0.037254,0.034782,0.0445,0.115218
46,-0.035945,-0.020355,0.013097,0.043335,0.028197,0.0151,0.121803
68,0.017606,-0.011199,0.026534,0.033756,0.045534,0.0190,0.104466


$ P_{transmission} = e^{-\frac{x}{\lambda}} $ <br>

x is thickness and $\lambda$ is attenuation length
<br> $\lambda = \frac{1}{\rho \cdot \mu} $ <br>
$ P_{transmission} = e^{-\mu \cdot \rho \cdot x} $ <br>

$ \mu $ ($cm^2/g$) <br> $ \rho $ ($g/cm^3$) <br> $\lambda (cm)$ <br>
$ \mu$ from https://physics.nist.gov/PhysRefData/Xcom/html/xcom1.html
<br>
Tranmission factor from: https://www.ezag.com/home/products/isotope_products/isotrak_calibration_sources/reference_sources/gamma_sources/point_sources/#c601

# fMaterialsManager->AddMaterial("Acrylic","C5-H8-O2",1.18*g/cm3,"")
# ceramic: O7410-Na300-Ca60-Mg10-Al600-Si1600-Pb4


![image](beadcomposition.png)

In [69]:
#Probabality of scatering from the button
button_density = 1#.18        #g/cm^3 (from simulation data)
button_mu_pb   = 0.1976  # C5H8O2 for 47keV (0.047 MeV) gamma rays
bead_mu_pb     = 3.195E-01   #47 keV pb-21O for ceramic=> O7410Na300Ca60Mg10Al600Si1600Pb4

#trnasmission factors from manufacturer
transmission_pb=0.964   #47 keV pb-210

transmission_am=0.958   #60 keV am-241



#button_mu_am=0.1815    #60 keV am-241
#lambda calculation for acrylic button  (C5H802)
#fMaterialsManager->AddMaterial("Acrylic","C5-H8-O2",1.18*g/cm3,"");
#density is from oldmc




def transmission_prob(x,mu,rho):
    '''reutrns button transmission probability'''
    #return (1-np.exp(-1.0*x*mu*rho))
    return (np.exp(-1.0*x*mu*rho))

    
    
    

In [70]:
#adding the transmississin though button
df_less['button_transmission_prob']=df_less.apply(lambda x:transmission_prob(x['d_button'],button_mu_pb,button_density),axis=1)
df_less.head()

,x,y,z,r,z_new,d_bead,d_button,button_transmission_prob,bead_transmission_prob,prob_product
11,0.030745,0.010918,-0.021412,0.039024,0.037988,0.0594,0.112012,0.978110,0.894069,0.871021
24,0.005623,0.017734,0.011749,0.022004,0.046449,0.0347,0.103551,0.979746,0.936682,0.914337
39,-0.020889,0.029276,-0.009718,0.037254,0.034782,0.0445,0.115218,0.977490,0.919537,0.895163
46,-0.035945,-0.020355,0.013097,0.043335,0.028197,0.0151,0.121803,0.976219,0.971937,0.944722
68,0.017606,-0.011199,0.026534,0.033756,0.045534,0.0190,0.104466,0.979569,0.964818,0.941601


In [71]:
#creating the random number bewteen 0 and 1, if the prob_product is greater than random number add 1 else add 0
def random_assign(x):
    if x>np.random.rand():
        return 1
    return 0
    

In [75]:
#adding the transmississin though button
bead_density=2.0#g/cm^3  
df_less['bead_transmission_prob']=df_less.apply(lambda x:transmission_prob(x['d_bead'],bead_mu_pb,bead_density),axis=1)
df_less.head()

,x,y,z,r,z_new,d_bead,d_button,button_transmission_prob,bead_transmission_prob,prob_product
11,0.030745,0.010918,-0.021412,0.039024,0.037988,0.0594,0.112012,0.978110,0.962755,0.963371
24,0.005623,0.017734,0.011749,0.022004,0.046449,0.0347,0.103551,0.979746,0.978071,0.971095
39,-0.020889,0.029276,-0.009718,0.037254,0.034782,0.0445,0.115218,0.977490,0.971965,0.966435
46,-0.035945,-0.020355,0.013097,0.043335,0.028197,0.0151,0.121803,0.976219,0.990398,0.972458
68,0.017606,-0.011199,0.026534,0.033756,0.045534,0.0190,0.104466,0.979569,0.987932,0.974824


In [76]:
N_generated=df_less.shape[0]
#df_less['N_emitted']=df_less['button_transmission_prob']*df_less['bead_transmission_prob']*N_generated
df_less['prob_product']=df_less['button_transmission_prob']*df_less['bead_transmission_prob']
df_less.head()

,x,y,z,r,z_new,d_bead,d_button,button_transmission_prob,bead_transmission_prob,prob_product
11,0.030745,0.010918,-0.021412,0.039024,0.037988,0.0594,0.112012,0.978110,0.962755,0.941680
24,0.005623,0.017734,0.011749,0.022004,0.046449,0.0347,0.103551,0.979746,0.978071,0.958261
39,-0.020889,0.029276,-0.009718,0.037254,0.034782,0.0445,0.115218,0.977490,0.971965,0.950086
46,-0.035945,-0.020355,0.013097,0.043335,0.028197,0.0151,0.121803,0.976219,0.990398,0.966845
68,0.017606,-0.011199,0.026534,0.033756,0.045534,0.0190,0.104466,0.979569,0.987932,0.967748


In [77]:
avg_of_prob=round(df_less['prob_product'].mean(),3)
avg_of_prob

0.955

In [78]:
avg_random=round(df_less['prob_product'].apply(lambda x:random_assign(x)).mean(),3)
avg_random

0.955

# IGNORE BELOW THIS

In [28]:
def get_prob(bead_density):
    '''
    returns the net prob of transmission for the chosen density of bead
    '''
    df_less['bead_transmission_prob']=df_less.apply(lambda x:transmission_prob(x['d_bead'],bead_mu_pb,bead_density),axis=1)
    df_less['prob_product']=df_less['button_transmission_prob']*df_less['bead_transmission_prob']
    avg_prob=round(df_less['prob_product'].mean(),3)
    return [bead_density,avg_prob]

In [68]:
density_prob=[]
for i in np.arange(0,6,0.1):
    test=get_prob(i)
    density_prob.append(test)
    print(f"test :{test}")
df_less.head()
    

test :[0.0, 0.974]
test :[0.1, 0.973]
test :[0.2, 0.972]
test :[0.30000000000000004, 0.971]
test :[0.4, 0.969]
test :[0.5, 0.968]
test :[0.6000000000000001, 0.967]
test :[0.7000000000000001, 0.966]
test :[0.8, 0.965]
test :[0.9, 0.964]
test :[1.0, 0.963]
test :[1.1, 0.961]
test :[1.2000000000000002, 0.96]
test :[1.3, 0.959]
test :[1.4000000000000001, 0.958]
test :[1.5, 0.957]
test :[1.6, 0.956]
test :[1.7000000000000002, 0.955]
test :[1.8, 0.953]
test :[1.9000000000000001, 0.952]
test :[2.0, 0.951]
test :[2.1, 0.95]
test :[2.2, 0.949]
test :[2.3000000000000003, 0.948]
test :[2.4000000000000004, 0.947]
test :[2.5, 0.946]
test :[2.6, 0.944]
test :[2.7, 0.943]
test :[2.8000000000000003, 0.942]
test :[2.9000000000000004, 0.941]
test :[3.0, 0.94]
test :[3.1, 0.939]
test :[3.2, 0.938]
test :[3.3000000000000003, 0.937]
test :[3.4000000000000004, 0.936]
test :[3.5, 0.934]
test :[3.6, 0.933]
test :[3.7, 0.932]
test :[3.8000000000000003, 0.931]
test :[3.9000000000000004, 0.93]
test :[4.0, 0.929]

,x,y,z,r,z_new,d_bead,d_button,button_transmission_prob,bead_transmission_prob,prob_product
11,0.030745,0.010918,-0.021412,0.039024,0.037988,0.0594,0.112012,0.974221,0.894069,0.871021
24,0.005623,0.017734,0.011749,0.022004,0.046449,0.0347,0.103551,0.976144,0.936682,0.914337
39,-0.020889,0.029276,-0.009718,0.037254,0.034782,0.0445,0.115218,0.973492,0.919537,0.895163
46,-0.035945,-0.020355,0.013097,0.043335,0.028197,0.0151,0.121803,0.971999,0.971937,0.944722
68,0.017606,-0.011199,0.026534,0.033756,0.045534,0.0190,0.104466,0.975936,0.964818,0.941601


In [33]:
for i in np.linspace(0,1,0.1):
    test=get_prob(i)
    print(f"test :{test}")

TypeError: 'float' object cannot be interpreted as an integer

In [32]:
get_prob(0.3)

[0.3, 0.954]

In [81]:
avg_prob=round(df_less['prob_product'].mean(),3)
avg_prob

0.954

In [87]:
prob=get_prob(0.1)

In [88]:
prob

[0.3, 0.956]

In [89]:
for i in range(0.1,10.0,0.1):
    prob=get_prob(i)
    print(f"prob: {prob}")

TypeError: 'float' object cannot be interpreted as an integer